In [1]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import importlib
import PIL
from scipy.misc import imresize
import scipy
import os 

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss, yolo_eval
from yolo3.utils import get_random_data

c:\users\popeye\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [3]:
def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

In [4]:
def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

In [5]:
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
                #print('\n',annotation_lines[i],input_shape)                
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

In [6]:
def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

In [26]:
ss=800
annotation_path = 'train550.txt'
log_dir = 'logs/005/'
data_path='data/players2'
anchors_path='data/players2/yolo_anchors.txt'
classes_path='data/players2/player_classes.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
#YOLO_ANCHORS = np.array(((0.091667, 0.03125), (0.077778, 0.019531), (0.033228 , 0.027344)))
YOLO_ANCHORS = np.array(
    ((0.16389 , 0.038281), (0.15278, 0.036719), (0.10278 , 0.039063),
     (0.12917 , 0.027344), (0.094444, 0.029687),(0.091667  ,0.020313),
    ( 0.083333, 0.017969),(0.055556 , 0.014063),(0.034333, 0.01875)))
anchors = YOLO_ANCHORS*ss
anchors_number = YOLO_ANCHORS.shape[0]
anchors=anchors[[8,7,6,5,4,3,2,1,0],:]
#print(anchors)

In [27]:
val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(42)
np.random.shuffle(lines)
np.random.seed(None)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

In [28]:
images=[]
i=0
for j in range(num_val):
    linei=lines[j+num_train]
    linei = linei.split()    
    #print(linei[0])   
    im=PIL.Image.open(linei[0])
    if i==0:
        dims=np.array(im).shape
        im1=np.zeros((max(dims),max(dims),3))
        im1[0:dims[0],0:dims[1],:]=im
        im2 = imresize(im1,(ss, ss))
        images=np.expand_dims(im2, axis=0)
    if i>0:
        im1[0:dims[0],0:dims[1],:]=np.array(im)
        im2 = imresize(im1,(ss, ss))
        images = np.concatenate((images,np.expand_dims(im2, axis=0)),axis=0)
    i=i+1       
image_data = images
#image_data = image_data/255.        

c:\users\popeye\appdata\local\programs\python\python35\lib\site-packages\scipy\misc\pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if issubdtype(ts, int):
c:\users\popeye\appdata\local\programs\python\python35\lib\site-packages\scipy\misc\pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [29]:
input_shape = (ss,ss)
#model = create_model(input_shape, anchors, num_classes,
#            freeze_body=2, weights_path='logs/000/trained_weights_final.h5')

In [30]:
import yolome
importlib.reload(yolome)
from yolome import YOLO

In [31]:
yolo_class=YOLO(log_dir,data_path, ss)
print(yolo_class.model_path)
print('Score threshold is: ', yolo_class.score)
print('IOU threshold is: ', yolo_class.iou)
print('Class names are: ', yolo_class.class_names)
print('Anchors are: ', yolo_class.anchors)
print('Image size: ', yolo_class.model_image_size)

logs/005/trained_weights_final.h5 model, anchors, and classes loaded.
logs/005/trained_weights_final.h5
Score threshold is:  0.3
IOU threshold is:  0.45
Class names are:  ['player']
Anchors are:  [[ 27.4664  15.    ]
 [ 44.4448  11.2504]
 [ 66.6664  14.3752]
 [ 73.3336  16.2504]
 [ 75.5552  23.7496]
 [103.336   21.8752]
 [ 82.224   31.2504]
 [122.224   29.3752]
 [131.112   30.6248]]
Image size:  (800, 800)


In [32]:
out_path="output_images"
yolo_class.score=0.07
yolo_class.iou=0.45
if  not os.path.exists(out_path):
        os.makedirs(out_path)
model_boxes=[]        
for i in range(num_val):
    r_image,box,score = yolo_class.detect_image(image_data[i])    
    image = PIL.Image.fromarray(r_image)
    image.save(os.path.join(out_path,str(i)+'.png'))
    model_boxes.append(box)

Found 15 boxes for img
p 0.63 (638, 281) (671, 337)
p 0.81 (96, 239) (116, 293)
p 0.86 (624, 165) (646, 200)
p 0.90 (0, 129) (12, 169)
p 0.92 (82, 92) (100, 126)
p 0.92 (232, 283) (253, 333)
p 0.93 (289, 264) (310, 320)
p 0.93 (383, 201) (403, 252)
p 0.94 (298, 227) (314, 276)
p 0.94 (315, 50) (330, 86)
p 0.96 (529, 67) (547, 102)
p 0.97 (109, 180) (126, 227)
p 0.98 (42, 120) (61, 159)
p 0.98 (333, 153) (350, 192)
p 0.98 (317, 97) (334, 136)
1.7061738230843275
Found 19 boxes for img
p 0.51 (526, 107) (533, 129)
p 0.76 (393, 156) (404, 183)
p 0.82 (340, 105) (351, 128)
p 0.88 (453, 187) (466, 217)
p 0.88 (661, 131) (673, 153)
p 0.88 (160, 243) (183, 281)
p 0.88 (461, 183) (474, 212)
p 0.90 (293, 156) (305, 184)
p 0.90 (134, 186) (149, 217)
p 0.91 (544, 159) (557, 184)
p 0.92 (408, 167) (420, 190)
p 0.94 (630, 223) (647, 267)
p 0.94 (740, 165) (755, 193)
p 0.96 (700, 187) (718, 217)
p 0.97 (748, 198) (765, 228)
p 0.97 (207, 135) (220, 163)
p 0.97 (677, 271) (694, 312)
p 0.98 (589, 152) (

Found 18 boxes for img
p 0.61 (423, 347) (445, 411)
p 0.82 (451, 95) (464, 125)
p 0.88 (528, 121) (542, 154)
p 0.89 (781, 311) (800, 368)
p 0.90 (168, 254) (189, 310)
p 0.91 (34, 131) (50, 167)
p 0.93 (189, 183) (207, 224)
p 0.94 (254, 170) (271, 208)
p 0.95 (650, 192) (666, 235)
p 0.95 (582, 134) (597, 175)
p 0.95 (617, 152) (634, 190)
p 0.95 (313, 130) (330, 164)
p 0.96 (431, 170) (446, 206)
p 0.96 (462, 117) (477, 149)
p 0.96 (346, 146) (361, 180)
p 0.97 (472, 153) (485, 191)
p 0.97 (189, 148) (203, 181)
p 0.97 (537, 222) (554, 266)
0.17003147052014356
Found 17 boxes for img
p 0.50 (739, 163) (762, 201)
p 0.51 (687, 350) (715, 393)
p 0.58 (241, 271) (268, 309)
p 0.64 (532, 251) (551, 299)
p 0.68 (566, 283) (590, 329)
p 0.74 (619, 357) (641, 407)
p 0.75 (561, 193) (578, 233)
p 0.75 (410, 251) (429, 288)
p 0.84 (327, 305) (348, 346)
p 0.87 (485, 59) (496, 94)
p 0.88 (599, 213) (621, 260)
p 0.91 (416, 273) (436, 311)
p 0.91 (554, 261) (571, 304)
p 0.93 (291, 304) (314, 346)
p 0.94 (381

Found 21 boxes for img
p 0.34 (583, 191) (598, 230)
p 0.40 (282, 413) (305, 446)
p 0.62 (471, 394) (497, 447)
p 0.63 (65, 349) (86, 407)
p 0.81 (587, 187) (602, 222)
p 0.88 (100, 278) (121, 327)
p 0.89 (470, 179) (485, 210)
p 0.89 (569, 207) (589, 245)
p 0.89 (600, 203) (621, 238)
p 0.90 (634, 91) (649, 120)
p 0.91 (467, 247) (483, 290)
p 0.92 (689, 191) (709, 228)
p 0.92 (152, 260) (172, 305)
p 0.93 (459, 177) (476, 211)
p 0.94 (467, 279) (484, 320)
p 0.94 (210, 192) (226, 228)
p 0.95 (251, 192) (268, 227)
p 0.95 (426, 194) (441, 232)
p 0.96 (334, 141) (350, 172)
p 0.97 (499, 110) (514, 142)
p 0.98 (371, 215) (388, 255)
0.18647059481031647
Found 18 boxes for img
p 0.67 (103, 414) (129, 443)
p 0.67 (32, 328) (53, 380)
p 0.71 (27, 249) (49, 300)
p 0.78 (204, 211) (222, 258)
p 0.81 (364, 349) (385, 399)
p 0.87 (287, 148) (301, 178)
p 0.87 (290, 163) (306, 200)
p 0.88 (440, 155) (460, 184)
p 0.93 (388, 250) (407, 291)
p 0.93 (183, 145) (199, 181)
p 0.94 (194, 208) (213, 250)
p 0.95 (73, 1

Found 18 boxes for img
p 0.45 (672, 348) (706, 404)
p 0.68 (82, 351) (114, 410)
p 0.87 (96, 235) (116, 279)
p 0.89 (328, 149) (348, 182)
p 0.91 (289, 263) (306, 308)
p 0.91 (278, 220) (293, 256)
p 0.92 (359, 174) (374, 203)
p 0.94 (552, 271) (572, 315)
p 0.94 (291, 220) (310, 256)
p 0.94 (379, 236) (400, 269)
p 0.95 (691, 196) (707, 231)
p 0.96 (237, 242) (253, 283)
p 0.96 (506, 216) (525, 253)
p 0.97 (371, 202) (390, 235)
p 0.97 (469, 158) (482, 189)
p 0.98 (532, 143) (548, 171)
p 0.98 (573, 209) (591, 245)
p 0.98 (251, 157) (272, 188)
0.1902327129475907
Found 11 boxes for img
p 0.42 (628, 246) (648, 308)
p 0.68 (513, 313) (545, 381)
p 0.71 (394, 324) (414, 385)
p 0.79 (50, 197) (72, 258)
p 0.82 (265, 278) (285, 335)
p 0.89 (109, 128) (124, 172)
p 0.92 (427, 242) (448, 295)
p 0.93 (410, 96) (428, 139)
p 0.93 (138, 113) (155, 155)
p 0.94 (251, 141) (270, 189)
p 0.98 (176, 370) (215, 439)
0.17415467996693224


In [33]:
i=0
all_boxes=[]
for j in range(num_val):
    linei=lines[j+num_train]
    linei = linei.split()    
    boxj=np.zeros((len(linei)-1,4))
    for j in range(len(linei)-1):
        linebox=linei[j+1]
        lineboxj=np.fromstring(linebox,dtype=int, sep=',')    
        boxj[j,:4]=lineboxj[:4]    
    all_boxes.append(boxj)
    i=i+1       

In [34]:
def iou(box1,box2):
    area1=(box1[2]-box1[0])*(box1[3]-box1[1])
    area2=(box2[2]-box2[0])*(box2[3]-box2[1])
    area3=(min(box1[2],box2[2]) - max(box1[0],box2[0]) ) * (min(box1[3],box2[3]) - max(box1[1],box2[1]) )
    # find iou between two boxes
    return area3/(area1+area2-area3)

In [35]:
def eval_boxes(box1,box2,ss):
    # box1 and box2 are my rectangles and default regtangles for players
    # the function compares them and outputs matched rectanges as well as their iou-s
    #box1=all_boxes[11] # This would rectangles for 11th image in val data
    #box2=model_boxes[11] # This would be rectgangles for 11th image detected by YOLO
    from munkres import Munkres        
    
    box2=box2*1280/ss # adjust for size difference
    box2=box2[:,[1,0,3,2]] #readjust order, first x, than y
    dd = scipy.spatial.distance.cdist(box2,box1) # appro distance bewtween rectangles. Here it gets 4D distance betwee vertices
    diag = np.sqrt(np.square(box1[:,2]-box1[:,0])+np.square(box1[:,3]-box1[:,1])) # diagonal of the test data boxes
    diag=diag.reshape(1, len(diag))   
    dd_log=dd<0.8*diag
    
    m = Munkres()
    infdist=10000
    ddold=dd   
    dd[dd_log==False]=infdist
    dd1=m.pad_matrix(dd.tolist(),infdist)
    #print(len(dd1),len(box2), len(box1))
    indexes = m.compute(dd1)
    
    mb=box2.shape[0]
    vb=box1.shape[0]
    ious=np.zeros((min(mb,vb)))
    
    det_matched=-1
    for row, column in indexes:
        if dd1[row][column]<100: # real distance
            #print(row, column, dd1[row][column],iou(box2[row,:],box1[column,:]))
            if iou(box2[row,:],box1[column,:])>0.3:                
                det_matched=det_matched+1   
                ious[det_matched]=iou(box2[row,:],box1[column,:])
    #print(dd.astype(int))            
    #print(indexes)
    #print(box2)
    #print(box1)
    ious=ious[ious>0]                
    return det_matched+1, mb,vb, ious                                    

In [36]:
det_matched = 0
det_total = 0
det_total_real = 0 
ious=np.zeros(1000)
ious_i=0
for j in range(num_val):
    out1,out2,out3,out4 = eval_boxes(all_boxes[j],model_boxes[j],ss)
    det_matched += out1
    det_total +=out2
    det_total_real += out3
    ious[ious_i:ious_i+len(out4)]=out4
    ious_i+=len(out4)
    #print(j, out1,out2,out3)

In [37]:
print('det_matched = ', det_matched)
print('det_total = ', det_total)
print('det_total_real = ', det_total_real)

det_matched =  815
det_total =  830
det_total_real =  854


In [38]:
print(np.mean(ious[ious>0]))

0.7295284944685535


In [20]:
#row_dist = np.sum(dd_log,axis=0)
#col_dist = np.sum(dd_log,axis=1)
# print(row_dist.shape,row_dist,col_dist.shape, col_dist)
#withwhom=np.argmax(dd_log,axis=0); # with which model box we connect
#withwhomsum=col_dist[withwhom]
#print(withwhom,withwhomsum)
#match=(row_dist==1) & (withwhomsum==1)
#print(match)
#matchlist=np.zeros((dd.shape[1],1))
#matchlist[match]=withwhom[match].reshape(np.sum(match),1)
#print(match,'\n',matchlist)
#dd_log[:,match]=False
#row_dist = np.sum(dd_log,axis=0)
#col_dist = np.sum(dd_log,axis=1)
#print(row_dist.shape,row_dist,col_dist.shape, col_dist)
#print(dd_log.astype(int))